# 手写藏文数字数据集分类

## 藏文数字


![Image Name](https://cdn.kesci.com/upload/image/piy7a8zo9h.jpg?imageView2/0/w/320/h/320)


藏文数字与阿拉伯数字一一对应，但写法不同[[1]](https://zh.wikipedia.org/wiki/%E8%97%8F%E6%96%87)。

|藏文数字|阿拉伯数字|
|---|---|
|༠ | 0|
|༡ | 1|
|༢ | 2|
|༣ | 3|
|༤ | 4|
|༥ | 5|
|༦ | 6|
|༧ | 7|
|༨ | 8|
|༩ | 9|


## 关于Input数据的再处理
由于**.npz** 数据是**有序**的图像及类别构成的，这对于图像分类训练来说是不利的，所以为了要保证**平衡样**本和**随机分布**的条件，我们需要做一些数据再处理。这在解决现实场景中图像相关问题的数据准备也是有帮助的。

不过通常的做法是在读取图片列表的时候就完成这些步骤，不过如果当你发现你已经完成了图片到数组数据转化，而忘记了保证训练集与测试集的样本类别的分布情况，那么你可以参考下面👇的数据处理方法，相信会给你一些启发与帮助。


## 使用两种模型来进行图像分类
在这篇notebook中，我们会尝试两种模型来进行图像分类。在这里，FCNN模型的训练时间在2分钟以内，CNN模型的训练时间在20分钟以内，所以不用担心会等很久。
- Fully Connected Neural Network 全连接神经网络
- Convolutional Neural Network 卷积神经网络

**Tips**

K-Lab提供了查看项目目录的功能，更简洁明了的向用户展示项目的框架结构，点击目录章节即可跳转到对应的章节。（点击左侧的目录列表图标即可）

![Image Name](https://cdn.kesci.com/upload/image/piy6i27du8.png?imageView2/0/w/1280/h/1280)


In [1]:
# 查看当前挂载的数据集目录
!ls /home/kesci/input/

TibetanMnist7208


In [2]:
# 显示cell运行时长
%load_ext klab-autotime

## 1.读取数据
### 1.1.从.NPZ文件中读取

In [3]:
from tensorflow import keras
from keras import utils
import matplotlib.pyplot as plt

time: 1.47 s


Using TensorFlow backend.


In [2]:
import numpy as np

img_rows,img_cols = 28,28

data = np.load('/home/kesci/input/TibetanMnist7208/TibetanMNIST.npz')

image_data, label_data= data['image'], data['label']

print('image数据：',image_data.shape)
print('Label数据：',label_data.shape)

image数据： (17768, 28, 28)
Label数据： (17768,)


### 1.2. 查看图片数组的类别分布
通过查看每个类别的所对应的下标，可以看到每个类别的图片是有序分布的，这对于我们后续拆分训练集与测试集训练模型是有弊端的。所以，我们需要让图像及对应类别样本**平衡**的**随机**分布，保证在训练集与测试集中涵盖了所有的类别数据，且每个类别的**数量相当**。

In [5]:
idx_list = {}
for i in range(10):
    idx_list[i] = np.where(label_data == i) # return tuple dtype (rows indices, column indices)

for label,idxs in idx_list.items():
    print('类别：',label,'数量：',len(idxs[0]), '\n对应index:',idxs)

类别： 0 数量： 1996 
对应index: (array([   0,    1,    2, ..., 1993, 1994, 1995]),)
类别： 1 数量： 2134 
对应index: (array([1996, 1997, 1998, ..., 4127, 4128, 4129]),)
类别： 2 数量： 1872 
对应index: (array([4130, 4131, 4132, ..., 5999, 6000, 6001]),)
类别： 3 数量： 1816 
对应index: (array([6002, 6003, 6004, ..., 7815, 7816, 7817]),)
类别： 4 数量： 1967 
对应index: (array([7818, 7819, 7820, ..., 9782, 9783, 9784]),)
类别： 5 数量： 1350 
对应index: (array([ 9785,  9786,  9787, ..., 11132, 11133, 11134]),)
类别： 6 数量： 1618 
对应index: (array([11135, 11136, 11137, ..., 12750, 12751, 12752]),)
类别： 7 数量： 1302 
对应index: (array([12753, 12754, 12755, ..., 14052, 14053, 14054]),)
类别： 8 数量： 1642 
对应index: (array([14055, 14056, 14057, ..., 15694, 15695, 15696]),)
类别： 9 数量： 2071 
对应index: (array([15697, 15698, 15699, ..., 17765, 17766, 17767]),)
time: 10.9 ms


### 1.3. 平衡样本 + 随机排序
注意：选取的是原样本列表的下标index
- **Step1：每个类别随机选出20%的样本作为测试集样本，其余的用于训练集样本。 `测试集:训练集 = 8 ：2`**
- **Step2：将训练集与测试集分别shuffle(X与y以同样的order进行shuffle)**

#### 1.3.1. Step1 每个类别随机选出20%的样本作为测试集样本，其余的用于训练集样本
**测试集的下标选取(随机选取20%)**

In [6]:
import random

selected_test_sample_indices = {}
for label in range(10):
    selected_test_sample_indices[label] = random.sample(set(idx_list[label][0]), int(len(idx_list[label][0])*0.2))
    

time: 7.18 ms


In [7]:
for label in range(10):
    # print(selected_test_sample_indices[label])
    print('类别',label,'选取测试集样本量：',len(selected_test_sample_indices[label]),'原本样本量：',len(idx_list[label][0]))
    

类别 0 选取测试集样本量： 399 原本样本量： 1996
类别 1 选取测试集样本量： 426 原本样本量： 2134
类别 2 选取测试集样本量： 374 原本样本量： 1872
类别 3 选取测试集样本量： 363 原本样本量： 1816
类别 4 选取测试集样本量： 393 原本样本量： 1967
类别 5 选取测试集样本量： 270 原本样本量： 1350
类别 6 选取测试集样本量： 323 原本样本量： 1618
类别 7 选取测试集样本量： 260 原本样本量： 1302
类别 8 选取测试集样本量： 328 原本样本量： 1642
类别 9 选取测试集样本量： 414 原本样本量： 2071
time: 7.77 ms


**训练集的下标选取（剩余80%）**

In [8]:
selected_train_sample_indicies = {}

for label in range(10):
    selected_train_sample_indicies[label] = list(set(idx_list[label][0])- set(selected_test_sample_indices[label]))
    

time: 3.87 ms


In [9]:
for label in range(10):
    # print(selected_train_sample_indicies[label])
    print('类别',label,'选训练集集样本量：',len(selected_train_sample_indicies[label]),'原本样本量：',len(idx_list[label][0]))
    

类别 0 选训练集集样本量： 1597 原本样本量： 1996
类别 1 选训练集集样本量： 1708 原本样本量： 2134
类别 2 选训练集集样本量： 1498 原本样本量： 1872
类别 3 选训练集集样本量： 1453 原本样本量： 1816
类别 4 选训练集集样本量： 1574 原本样本量： 1967
类别 5 选训练集集样本量： 1080 原本样本量： 1350
类别 6 选训练集集样本量： 1295 原本样本量： 1618
类别 7 选训练集集样本量： 1042 原本样本量： 1302
类别 8 选训练集集样本量： 1314 原本样本量： 1642
类别 9 选训练集集样本量： 1657 原本样本量： 2071
time: 7.13 ms


#### Step 2: 将训练集与测试集分别shuffle
**对于训练集与测试集下标字典，将每个类别选取出的下标整合成一个列表**


In [10]:
train_data_indicies, test_data_indicies = [],[]

# 训练集的所有类别下标
for label,indicies in selected_train_sample_indicies.items():
    train_data_indicies = train_data_indicies + indicies # merge 2 list
# 测试集的所有类别下标
for label,indicies in selected_test_sample_indices.items():
    test_data_indicies = test_data_indicies + indicies

# 打乱顺序(随机排列)
random.shuffle(train_data_indicies)
random.shuffle(test_data_indicies)

print('训练集样本量：',len(train_data_indicies), '测试集样本量：',len(test_data_indicies))

训练集样本量： 14218 测试集样本量： 3550
time: 19 ms


### 1.4. 构建测试集与训练集
**根据1.3准备好顺序的下标列表，依次应用到图像及类别数据上。**
- 让训练集中的图像与label以同样的顺序构成新的列表
- 让测试集中的图像与label以同样的顺序构成新的列表

In [11]:
y_train_data = np.array([label_data[idx] for idx in train_data_indicies])
X_train_data = np.array([image_data[idx] for idx in train_data_indicies])

y_test_data = np.array([label_data[idx] for idx in test_data_indicies])
X_test_data = np.array([image_data[idx] for idx in test_data_indicies])

time: 86.7 ms


### 1.4. 查看样例图
当前已经实现了平衡分布与随机排列的两个条件，现在随机抽查两个样本来看下图片，及对应类别label是否正确。

In [13]:
plt.imshow(X_test_data[2541], cmap = 'gray')
plt.title('Class '+ str(y_test_data[2541]))
plt.show()

<Figure size 432x288 with 1 Axes>

time: 179 ms


In [15]:
plt.imshow(X_train_data[2], cmap = 'gray')
plt.title('Class '+ str(y_train_data[2]))
plt.show()

<Figure size 432x288 with 1 Axes>

time: 228 ms


In [ ]:
# split =int(0.8*len(image_data))
# print("切分值：",split)

# X_train_data = image_data[:split]
# X_test_data = image_data[split:]
# y_train_data = label_data[:split]
# y_test_data = label_data[split:]

# print('original训练集X：',X_train_data.shape,'original训练集y：',y_train_data.shape)
# print('original测试集X',X_test_data.shape,'original测试集y：',y_test_data.shape)

## 2.数据预处理
### 2.1. label数据进行**one-hot encoding**处理

In [16]:
number_of_classes = 10

y_train = utils.to_categorical(y_train_data, number_of_classes) # one-hot encoding
y_test = utils.to_categorical(y_test_data, number_of_classes) # one-hot encoding

time: 2.64 ms


In [23]:
# 查看一个类别样本
print('对应类别为7\n',y_train[12112])

对应类别为7
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
time: 2.19 ms


### 2.2. reshape图片特征array
这是用于模型1全连接神经网络的输入所用

In [24]:
# Reshape 特征样本
X_train = X_train_data.reshape(X_train_data.shape[0], img_rows*img_cols)
X_test = X_test_data.reshape(X_test_data.shape[0], img_rows*img_cols)

print('reshaped 训练集X：','reshaped 训练集y：',X_train.shape,y_train.shape)
print('reshaped 测试集X','reshaped 测试集y：',X_test.shape,y_test.shape)

reshaped 训练集X： reshaped 训练集y： (14218, 784) (14218, 10)
reshaped 测试集X reshaped 测试集y： (3550, 784) (3550, 10)
time: 82.1 ms


### 2.2. 归一化图像数据
- **rescale 取值范围 `[0,255]`成`[0,1]`**

In [25]:
X_train = X_train / 255
X_test = X_test / 255

time: 33.6 ms


## 3.训练模型
## 3.1. 模型1：全连接神经网络
- epochs = 30
	- 运行时间1min29s
	- 测试集Accuracy: 82.06%
	- 测试集loss:2.16；
- epochs = 40
	- 运行时间1min58秒
	- 测试集loss:1.60
	- 测试集Accuracy:85.83%

In [31]:
def FCNN(epochs):
    # create model
    model = keras.Sequential()
    model.add(keras.layers.Dense(784, input_dim=784, kernel_initializer='normal', activation= 'tanh'))
    model.add(keras.layers.Dense(512, kernel_initializer='normal', activation= 'tanh'))
    model.add(keras.layers.Dense(512, kernel_initializer='normal', activation= 'tanh'))
    model.add(keras.layers.Dense(10, kernel_initializer='normal', activation= 'softmax'))
    
    # Compile model 
    # 损失函：数交叉函数熵
    # 优化器：adam
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # Fit the model
    model.fit(X_train, 
            y_train, 
            epochs=epochs, 
            batch_size=200, 
            verbose=1, 
            validation_data=(X_test,y_test)
            )
    return model

time: 3.46 ms


**Epochs = 30**

In [32]:
epochs = 30
model1 = FCNN(epochs=epochs)

Train on 14218 samples, validate on 3550 samples
Epoch 1/30
14218/14218 [==============================] - 3s 237us/step - loss: 11.8595 - acc: 0.1814 - val_loss: 10.0464 - val_acc: 0.2851
Epoch 2/30
14218/14218 [==============================] - 3s 210us/step - loss: 8.4464 - acc: 0.3913 - val_loss: 7.6192 - val_acc: 0.4420
Epoch 3/30
14218/14218 [==============================] - 3s 207us/step - loss: 6.6349 - acc: 0.5067 - val_loss: 6.2335 - val_acc: 0.5344
Epoch 4/30
14218/14218 [==============================] - 3s 206us/step - loss: 5.5854 - acc: 0.5722 - val_loss: 5.5333 - val_acc: 0.5845
Epoch 5/30
14218/14218 [==============================] - 3s 209us/step - loss: 4.6863 - acc: 0.6307 - val_loss: 4.7545 - val_acc: 0.6223
Epoch 6/30
14218/14218 [==============================] - 3s 208us/step - loss: 4.0149 - acc: 0.6729 - val_loss: 4.2106 - val_acc: 0.6566
Epoch 7/30
14218/14218 [==============================] - 3s 208us/step - loss: 3.5094 - acc: 0.7097 - val_loss: 3.8513 -

In [42]:
score1 = model1.evaluate(X_test,y_test, verbose=0)
print('Epochs:{0}, 测试集loss:{1:.02f}, 测试集Accuracy:{2:.2%}'.format(epochs,score1[0],score1[1]))

Epochs:30, 测试集loss:2.16, 测试集Accuracy:82.06%
time: 411 ms


**Epochs = 40**

In [44]:
epochs = 40
model2 = FCNN(epochs=epochs)

Train on 14218 samples, validate on 3550 samples
Epoch 1/40
14218/14218 [==============================] - 3s 240us/step - loss: 11.9571 - acc: 0.1811 - val_loss: 10.1538 - val_acc: 0.2893
Epoch 2/40
14218/14218 [==============================] - 3s 206us/step - loss: 8.6996 - acc: 0.3744 - val_loss: 7.4362 - val_acc: 0.4482
Epoch 3/40
14218/14218 [==============================] - 3s 207us/step - loss: 6.6314 - acc: 0.5025 - val_loss: 5.9358 - val_acc: 0.5558
Epoch 4/40
14218/14218 [==============================] - 3s 205us/step - loss: 5.4291 - acc: 0.5857 - val_loss: 5.0473 - val_acc: 0.6042
Epoch 5/40
14218/14218 [==============================] - 3s 207us/step - loss: 4.6773 - acc: 0.6366 - val_loss: 4.5936 - val_acc: 0.6470
Epoch 6/40
14218/14218 [==============================] - 3s 206us/step - loss: 4.2236 - acc: 0.6678 - val_loss: 4.2520 - val_acc: 0.6623
Epoch 7/40
14218/14218 [==============================] - 3s 205us/step - loss: 3.7653 - acc: 0.6987 - val_loss: 3.9279 -

In [46]:
score2 = model2.evaluate(X_test,y_test, verbose=0)
print('Epochs:{0}, 测试集loss:{1:.02f}, 测试集Accuracy:{2:.2%}'.format(epochs,score2[0],score2[1]))

Epochs:40, 测试集loss:1.60, 测试集Accuracy:85.83%
time: 408 ms


在一定程度下，iterate的次数的增加可以提升准确度，但这并不代表无限的增加循环次数，准确率就一定会更高，很有可能会出现过拟合。
全连接神经网络的能力是有限的，是**不适合做图像识别任务**的。因为通过梯度下降训练它很难，它的梯度很难传递超过三层，我们是得不到一个很深的全连接神经网络。

## 3.2. 模型2：卷积神经网络（CNN）
CNN是很适合做图像识别的，它可以尽可能的保留图像中重要的参数，舍去大量不必要的参数，从而达到更好的学习效果。接下来我们尝试使用CNN来进行图片分类
- epochs = 5, batchsize = 64*3 = 192
- 运行时间15分43秒
- 测试集loss: 0.08
- 测试集accuracy: 97.55%

In [3]:
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

input_shape = (img_rows, img_cols, 1)

### 3.2.1. 数据准备
#### 3.2.2. 图像数据
- 将图像数据reshape成（X_train_data.shape[0],28,28,1）
- 归一化图像数据

In [48]:
X_train = X_train_data.reshape(X_train_data.shape[0], img_rows,img_cols,1)
X_test = X_test_data.reshape(X_test_data.shape[0], img_rows,img_cols,1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

print('训练集X:',X_train.shape, '\n测试集X:',X_test.shape)

训练集X: (14218, 28, 28, 1) 
测试集X: (3550, 28, 28, 1)
time: 45 ms


#### 3.2.3. label数据
- one-hot encoding处理，这个在之前的过程中已经完成，所以这里就不重复了。
- 比如：y_train[0] = array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])， 表示label为1。


### 3.2.2. 构建模型
#### CNN网络构建的3个步骤：
1. **卷积 Convolution**
	- **关于卷积层输出值的计算动图如下所示**
	假设图像是5x5，使用一个3x3的filter进行卷积，得到一个3x3的feature map。
	![Image Name](https://cdn.kesci.com/upload/image/piy82itx4i.gif?imageView2/0/w/320/h/320)
2. **激活 Activation**
3. **池化 Pooling**
**重复1-3步骤来添加更多的隐藏层（hidden layers**）
4. **然后构建一个全连接网络，让CNN能进行样本分类**






In [4]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(28,28,1)))
model.add(BatchNormalization(axis=-1)) # normalize
model.add(Activation('relu'))

model.add(Conv2D(32, kernel_size=(3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten()) # 展开成1-D的阵列

# Fully Connected Layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(10)) # 10: num_classes

model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
batch_normalization_1 (Batch (None, 26, 26, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 24, 24, 32)        128       
_________________________________________________________________
activation_2 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 22, 22, 64)        18496     
__________

In [50]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

time: 44 ms


#### 3.2.3.训练模型

In [51]:
epochs = 5

model.fit(X_train, y_train,
          batch_size=192,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test), )

Train on 14218 samples, validate on 3550 samples
Epoch 1/5
14218/14218 [==============================] - 190s 13ms/step - loss: 0.1960 - acc: 0.9372 - val_loss: 0.0980 - val_acc: 0.9713
Epoch 2/5
14218/14218 [==============================] - 188s 13ms/step - loss: 0.0402 - acc: 0.9875 - val_loss: 0.0839 - val_acc: 0.9727
Epoch 3/5
14218/14218 [==============================] - 188s 13ms/step - loss: 0.0216 - acc: 0.9949 - val_loss: 0.0661 - val_acc: 0.9797
Epoch 4/5
14218/14218 [==============================] - 188s 13ms/step - loss: 0.0123 - acc: 0.9971 - val_loss: 0.0784 - val_acc: 0.9786
Epoch 5/5
14218/14218 [==============================] - 188s 13ms/step - loss: 0.0095 - acc: 0.9974 - val_loss: 0.0823 - val_acc: 0.9755


time: 15min 43s


#### 3.2.4.模型效果

In [53]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Epochs:{0}, 测试集loss:{1:.02f}, 测试集Accuracy:{2:.2%}'.format(epochs,score[0],score[1]))

Epochs:5, 测试集loss:0.08, 测试集Accuracy:97.55%
time: 15.2 s


我们用这个简单的CNN模型得到了97.55%的准确率，想要提升准确率，可以尝试数据增量（data augmentation）,或者模型集成（ensembling）等。这些就等你来探索啦~